In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [21]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
# spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                dcc.Dropdown(id='site-dropdown',options=[{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()] + [{'label': 'All Sites', 'value': 'All_Sites'}],
                                             
                                            value='All_Sites',searchable=True),
                                html.Br(),

                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=min_payload,max=max_payload,step=1000,
                                               value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart','figure'),
             Input('site-dropdown','value'))

def get_pie(entered_site):
    filtereddf=spacex_df
    if entered_site == 'All_Sites':
        result=filtereddf['class'].value_counts().reset_index()
        fig=px.pie(result,values='count',names='class',title="Pie Graph for all Launch Site")
        return fig
    else:
        filtereddf=filtereddf[filtereddf['Launch Site']==entered_site]
        result=filtereddf['class'].value_counts().reset_index()
        fig=px.pie(result,values='count',names='class',title=f"Pie Graph for all {entered_site} Site")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart','figure'),
             [Input('site-dropdown','value'),
             Input('payload-slider','value')])


def get_scatter(entered_site,payload):
    filtereddf=spacex_df
    minval,maxval=payload
    if entered_site == 'All_Sites':
        filtereddf = filtereddf[(filtereddf['Payload Mass (kg)'] >= minval) & (filtereddf['Payload Mass (kg)'] <= maxval)]
        fig=px.scatter(filtereddf,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig
    else:
        filtereddf=filtereddf[filtereddf['Launch Site']==entered_site]
        filtereddf = filtereddf[(filtereddf['Payload Mass (kg)'] >= minval) & (filtereddf['Payload Mass (kg)'] <= maxval)]
        fig=px.scatter(filtereddf,x='Payload Mass (kg)',y='class',color='Booster Version Category')
        return fig        





# Run the app
if __name__ == '__main__':
    app.run_server(port=8052)
